<a href="https://colab.research.google.com/github/marga0428ri/stock-ai/blob/main/stock_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

In [2]:

import yfinance as yf
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical

In [3]:

ticker = "AAPL"
df = yf.download(ticker, start="2015-01-01", end="2024-01-01")

df = df[["Close", "Volume"]]
df.dropna(inplace=True)

/tmp/ipython-input-3996520735.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2015-01-01", end="2024-01-01")
[*********************100%***********************]  1 of 1 completed


In [5]:

# 価格変化率
df["r"] = df["Close"].pct_change()

# 出来高異常（20日平均との差）
df["v"] = df["Volume"] / df["Volume"].rolling(20).mean()

# トレンド（短期 - 長期移動平均）
df["s"] = df["Close"].rolling(5).mean() - df["Close"].rolling(20).mean()

df.dropna(inplace=True)

In [6]:

def label_direction(x, threshold=0.01):
    if x > threshold:
        return 0  # 上昇
    elif x < -threshold:
        return 2  # 下落
    else:
        return 1  # 横ばい

df["future_return"] = df["Close"].pct_change().shift(-1)
df["label"] = df["future_return"].apply(label_direction)

df.dropna(inplace=True)

In [7]:

FEATURES = ["r", "v", "s"]
SEQ_LEN = 20

X, y = [], []

for i in range(SEQ_LEN, len(df)):
    X.append(df[FEATURES].iloc[i-SEQ_LEN:i].values)
    y.append(df["label"].iloc[i])

X = np.array(X)
y = to_categorical(y, num_classes=3)

In [8]:
scaler = StandardScaler()

for i in range(X.shape[1]):
    X[:, i, :] = scaler.fit_transform(X[:, i, :])

In [9]:
split = int(len(X) * 0.8)

X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [10]:
model = Sequential([
    LSTM(32, input_shape=(SEQ_LEN, X.shape[2])),
    Dense(3, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 32)             │         4,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │            99 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,707 (18.39 KB)

 Trainable params: 4,707 (18.39 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.4903 - loss: 1.0708 - val_accuracy: 0.5191 - val_loss: 1.0274
Epoch 2/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.5614 - loss: 0.9727 - val_accuracy: 0.5213 - val_loss: 1.0281
Epoch 3/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.5545 - loss: 0.9680 - val_accuracy: 0.4719 - val_loss: 1.0696
Epoch 4/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5628 - loss: 0.9510 - val_accuracy: 0.4944 - val_loss: 1.0309
Epoch 5/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5558 - loss: 0.9584 - val_accuracy: 0.4989 - val_loss: 1.0443
Epoch 6/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5712 - loss: 0.9538 - val_accuracy: 0.4831 - val_loss: 1.0296
Epoch 7/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5651 - loss: 0.9565 - val_accuracy: 0.4674 - val_loss: 1.0480
Epoch 8/20
56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5510 - loss: 0.9617 - val_accuracy: 0.4921 - v

In [12]:

pred = model.predict(X_test[-1].reshape(1, SEQ_LEN, X.shape[2]))

print("上昇確率:", pred[0][0])
print("横ばい確率:", pred[0][1])
print("下落確率:", pred[0][2])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step
上昇確率: 0.22389704
横ばい確率: 0.6449945
下落確率: 0.1311085


In [13]:
preds = model.predict(X_test)

np.mean(preds, axis=0)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


array([0.32182494, 0.43040007, 0.24777542], dtype=float32)

In [14]:
# 毎日実行される想定
df_new = yf.download("AAPL", period="1d")
df_new.to_csv("price_log.csv", mode="a", header=False)

/tmp/ipython-input-1010263313.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_new = yf.download("AAPL", period="1d")
[*********************100%***********************]  1 of 1 completed


In [18]:
# Colabで一度だけ
model.save("model.h5")

In [19]:
model.predict(X)

70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[0.2471122 , 0.6281399 , 0.12474798],
       [0.20750768, 0.6814632 , 0.11102909],
       [0.20341614, 0.6820332 , 0.11455064],
       ...,
       [0.28530738, 0.53888965, 0.17580302],
       [0.25461617, 0.5945262 , 0.15085767],
       [0.22389704, 0.6449945 , 0.1311085 ]], dtype=float32)

In [1]:
import yfinance as yf

def fetch(stock_code="AAPL"):
    df = yf.download(
        stock_code,
        start="2015-01-01",
        interval="1d"
    )
    return df

if __name__ == "__main__":
    data = fetch("AAPL")
    print(data.tail())

/tmp/ipython-input-609603499.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(
[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open    Volume
Ticker            AAPL        AAPL        AAPL        AAPL      AAPL
Date                                                                
2026-02-02  270.010010  270.489990  259.209991  260.029999  73913400
2026-02-03  269.480011  271.880005  267.609985  269.200012  64394700
2026-02-04  276.489990  278.950012  272.290009  272.290009  90545700
2026-02-05  275.910004  279.500000  273.230011  278.130005  52977400
2026-02-06  278.119995  280.910004  276.929993  277.119995  50420700


In [2]:
import yfinance as yf

def fetch(stock_code="AAPL"):
    df = yf.download(
        stock_code,
        start="2015-01-01",
        interval="1d",
        auto_adjust=True
    )
    return df

if __name__ == "__main__":
    data = fetch("AAPL")
    data.to_csv("AAPL_daily.csv")
    print("保存完了")

[*********************100%***********************]  1 of 1 completed

保存完了
